In [1]:
import pyflp
flp = '/Users/applelaptop/Downloads/oenis/oenis.flp'
project = pyflp.parse(flp)

patterns = [p for p in project.patterns]

In [2]:
serum = patterns[1]
notes = [n for n in serum.notes]

In [3]:
notes
# position:
# 96 = quarter note
# thus position 96 = 100 ticks
# position 48 should be midi tick 50
# 100/96 is the factor we want

[Note(key=A5, position=0, length=48, channel=2),
 Note(key=E6, position=0, length=48, channel=2),
 Note(key=A6, position=0, length=48, channel=2),
 Note(key=C#7, position=0, length=48, channel=2),
 Note(key=B6, position=192, length=48, channel=2),
 Note(key=G#5, position=192, length=48, channel=2),
 Note(key=D#6, position=192, length=48, channel=2),
 Note(key=F#7, position=192, length=48, channel=2),
 Note(key=G#7, position=288, length=48, channel=2),
 Note(key=C#6, position=384, length=48, channel=2),
 Note(key=G#6, position=384, length=48, channel=2),
 Note(key=C#7, position=384, length=48, channel=2),
 Note(key=E7, position=384, length=48, channel=2),
 Note(key=B5, position=480, length=48, channel=2),
 Note(key=E7, position=576, length=48, channel=2),
 Note(key=C#6, position=576, length=48, channel=2),
 Note(key=G#6, position=576, length=48, channel=2),
 Note(key=C#7, position=576, length=48, channel=2),
 Note(key=D#7, position=672, length=48, channel=2),
 Note(key=E6, position=672,

In [4]:
dat = []
for note in notes:
    d = {}
    d['note'] = note.key[:-1]
    d['octave'] = note.key[-1]
    d['position'] = note.position
    d['length'] = note.length
    dat.append(d)

In [5]:
notetonum = {
    'C': 0,
    'C#': 1,
    'D': 2,
    'D#': 3,
    'E': 4,
    'F': 5,
    'F#': 6,
    'G': 7,
    'G#': 8,
    'A': 9,
    'A#': 10,
    'B': 11
}

def tomidi(d):
    note = d['note']
    octave = int(d['octave'])*12
    num = notetonum[note]
    return num + octave


for d in dat:
    d['midinote'] = tomidi(d)
    absolute_time += int(d['position'])
    d['abstime'] = absolute_time


In [6]:
dat

[{'note': 'A',
  'octave': '5',
  'position': 0,
  'length': 48,
  'midinote': 69,
  'abstime': 0},
 {'note': 'E',
  'octave': '6',
  'position': 0,
  'length': 48,
  'midinote': 76,
  'abstime': 0},
 {'note': 'A',
  'octave': '6',
  'position': 0,
  'length': 48,
  'midinote': 81,
  'abstime': 0},
 {'note': 'C#',
  'octave': '7',
  'position': 0,
  'length': 48,
  'midinote': 85,
  'abstime': 0},
 {'note': 'B',
  'octave': '6',
  'position': 192,
  'length': 48,
  'midinote': 83,
  'abstime': 192},
 {'note': 'G#',
  'octave': '5',
  'position': 192,
  'length': 48,
  'midinote': 68,
  'abstime': 384},
 {'note': 'D#',
  'octave': '6',
  'position': 192,
  'length': 48,
  'midinote': 75,
  'abstime': 576},
 {'note': 'F#',
  'octave': '7',
  'position': 192,
  'length': 48,
  'midinote': 90,
  'abstime': 768},
 {'note': 'G#',
  'octave': '7',
  'position': 288,
  'length': 48,
  'midinote': 92,
  'abstime': 1056},
 {'note': 'C#',
  'octave': '6',
  'position': 384,
  'length': 48,
  'mid

In [60]:
import mido
def tomessage(d):
    return mido.Message('note_on', channel = 0, note = d['midinote'], velocity = 64, time = int(int(d['position'])*(100/96)))

In [61]:
msgs = [tomessage(d) for d in dat]

In [62]:
msgs

# times are delta times in ticks, how long to wait since last message. no idea to convert these into absolute time.

[Message('note_on', channel=0, note=69, velocity=64, time=0),
 Message('note_on', channel=0, note=76, velocity=64, time=0),
 Message('note_on', channel=0, note=81, velocity=64, time=0),
 Message('note_on', channel=0, note=85, velocity=64, time=0),
 Message('note_on', channel=0, note=83, velocity=64, time=200),
 Message('note_on', channel=0, note=68, velocity=64, time=200),
 Message('note_on', channel=0, note=75, velocity=64, time=200),
 Message('note_on', channel=0, note=90, velocity=64, time=200),
 Message('note_on', channel=0, note=92, velocity=64, time=300),
 Message('note_on', channel=0, note=73, velocity=64, time=400),
 Message('note_on', channel=0, note=80, velocity=64, time=400),
 Message('note_on', channel=0, note=85, velocity=64, time=400),
 Message('note_on', channel=0, note=88, velocity=64, time=400),
 Message('note_on', channel=0, note=71, velocity=64, time=500),
 Message('note_on', channel=0, note=88, velocity=64, time=600),
 Message('note_on', channel=0, note=73, velocity

In [63]:
track = mido.MidiTrack()
for msg in msgs:
    track.append(msg)

mid = mido.MidiFile()
mid.tracks.append(track)
from shortcuts.file_io import save_file_name

mid.save(save_file_name(ext = '.mid'))